# Flask Web Framework

## Hello World Example in Notebook

### Creating a Flask Application

In [2]:
!conda install flask --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::matplotlib==3.0.3=py37_1
done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.5

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - flask


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-2.10                  |         openblas           6 KB  conda-forge
    flask-1.1.1                |             py_0          69 KB  conda-forge
    itsdangerous-1.1.0         |             py_0          16 KB  conda-forge
    libblas-3.8.0              |      10_openblas           6 KB  conda-forge
    libcblas-3.8.0             |      10_openblas           6 KB  cond

In [7]:
%%python --bg --out app_out --err app_err # ipython magic to run this code in background

from flask import Flask
app = Flask(__name__) # creates Flask application

@app.route('/') # web page is accessable from url root 
@app.route('/index') # and index subdirectory
def index():
    return 'Hello World!'

# starts built-in development server - would block notebook kernel if not run in background
app.run() 

In [22]:
app_out.peek() # checking output (stdout) of Flask server

b' * Serving Flask app "<stdin>" (lazy loading)\n * Environment: production\n   WARNING: This is a development server. Do not use it in a production deployment.\n   Use a production WSGI server instead.\n * Debug mode: off\n'

In [23]:
app_err.peek() # checking stderr of Flask server

b' * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)\n'

Flask comes with a built-in web server suited for development purposes (not for production!).
It is rather an exotic use case to run such a server in a Jupyter Notebook. It would block the IPython kernel and make any further use of the notebook impossible.
Therefore an IPython magic command is used above to start a background process for the server.
This issue will not appear if you use Flask "normally", i.e. defining the application in .py file(s) and running the server with the Python interpreter.

### Accessing Web Page

In [1]:
import urllib.request

In [26]:
response = urllib.request.urlopen('http://127.0.0.1:5000/index')
response.read()

b'Hello World!'

# Flask Development Server Docker Container

As seen above, the Jupyter Notebook/ Lab is not really suited for developping web apps.
Better ways to run the Flask development server are:

1. run locally in a Python / Conda environment

2. run the Flask server in a dedicated Docker container

The first method is rather straight-forward. However, it requires a local Python version, in contrast to all other parts of this tutorial, which can be run completely in Docker containers.

In the following, the second method is discussed, both for having an environment completely running on Docker and (especially) for educational purposes.

## Idea

As an alternative to run the Flask bulit-in development server locally, the server could be executed in a Docker container.

The following container starts a Flask server and runs an app which is stored on a volume mount. If the app is changed, it is automatically synchronized with the container, no need to re-build the image, etc.

## Implementation

In [4]:
!cat ../flask/flask_devserver/Dockerfile

FROM continuumio/miniconda3
COPY environment.yaml .
RUN conda update conda --yes && \
	conda env update -f environment.yaml -n base && \
	conda clean -a

COPY start_server.py .

EXPOSE 5000
VOLUME /app

CMD ["python", "start_server.py"]


The official Miniconda 3 base image is used.

The required packages, defined in the file *environment.yaml* are installed with Conda into the base environment (no need to use a separate Conda environment inside a Docker container).

The *conda clean* command is used to delete cashes and temporary files to reduce image size.

The port 5000, on which the Flask server runs, is exposed. The directory containing the Flask app needs to be mounted as a volume.

In [5]:
!cat ../flask/flask_devserver/environment.yaml

name: flask_env
channels: 
    - default
    - conda-forge 
dependencies:
    - python=3.7
    - flask


*channels* define the Conda package sources in the listed priority. Here, the primary package source is the default channel and the secondary conda-forge.

The required packages for the Flask app are listed in *dependencies*. If no version number is given, the latest one which is compatible to the other packages is installed.

In [6]:
!cat ../flask/flask_devserver/start_server.py

from app import app # assumption: a Flask instance app is created in __init__.py of the app package

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0') # ip address for running in Docker container
    # default ip is localhost, this does not work inside a container


## Building Docker Image

### Manual Building

Using the following commands, the Docker image for the dev server is built.

    cd flask/flask_devserver
    # edit environment.yaml if additional packages are required
    [sudo] docker build -t flask_devserver:latest .
    
### Automatic Building using Docker Compose

The [docker-compose.yml](../docker-compose.yml) file already contains the commands to automatically build the flask dev server Docker image.

    [sudo] docker-compose up -d

Make sure to edit *environment.yaml* first if additional packages are required.

## Modification of Flask Code

Directly in the flask_devserver/app directory.

## Access of Flask Webpage

In [1]:
import urllib.request

In [2]:
response = urllib.request.urlopen('http://flask_devserver:5000/index')
response.read()

b'Hello, World!'

Access from Docker internal network

In [3]:
response = urllib.request.urlopen('http://192.168.1.24:25000') # ip address of docker host,
# probably different in your system (may be localhost)
response.read()

b'Hello, World!'

Access from outside Docker network

Alternative: access in Browser:

http://192.168.1.24:25000

# Flask Application

A great and very extensive Flask tutorial is given here:

https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world

Parts of the following content is taken (or at least inspired) from above.

The main difference of the following content to the tutorial above is that the latter uses SQLAlchemy as database ORM, whereas in the following example the No-SQL database MongoDB is used.